In [1]:
import sys
import os
from pathlib import Path
import pickle
import torch
import pprint
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
from utils import *
from configuration import *
import numpy as np

if not os.path.exists('./evaluation/'):
    os.makedirs('./evaluation/')

In [2]:
args = parser.parse_args([])
args.task = 'temporal'
args.metric = 'acc'
args.SoftEva = True
args = FormulateArgs(args)

In [3]:
valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
pprint.pprint(datainfo)

{'N_class': 3,
 'N_feature': 1,
 'N_test': 100,
 'N_time': 64,
 'N_train': 300,
 'N_valid': 100,
 'dataname': 'cbf'}


In [4]:
results = torch.zeros([80,10,2])

for seed in range(10):
    for ds in range(80):
        args.SEED = seed
        args.DATASET = ds

        valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
        test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')

        modelname = f"baseline_model_RNN_data_{args.DATASET:02d}_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
        print(modelname)
        
        if os.path.isfile(f'./models/{modelname}'):                
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            # model.UpdateArgs(args)

            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)

            for x,y in valid_loader:
                X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

            acc_valid = evaluator(model, X_valid, y_valid)
            acc_test   = evaluator(model, X_test,  y_test)

            results[ds,seed,0] = acc_valid
            results[ds,seed,1] = acc_test
        else:
            results[ds,seed,:] = float('nan')
            
# torch.save(results, f"./evaluation/result_data:{datainfo['dataname']}")


baseline_model_RNN_data_00_cbf_seed_00.model
baseline_model_RNN_data_01_chlorineconcentration_seed_00.model
baseline_model_RNN_data_02_cincecgtorso_seed_00.model
baseline_model_RNN_data_03_computers_seed_00.model
baseline_model_RNN_data_04_diatomsizereduction_seed_00.model
baseline_model_RNN_data_05_distalphalanxoutlineagegroup_seed_00.model
baseline_model_RNN_data_06_distalphalanxtw_seed_00.model
baseline_model_RNN_data_07_earthquakes_seed_00.model
baseline_model_RNN_data_08_ecg200_seed_00.model
baseline_model_RNN_data_09_ecg5000_seed_00.model
baseline_model_RNN_data_10_ecgfivedays_seed_00.model
baseline_model_RNN_data_11_eigenworms_seed_00.model
baseline_model_RNN_data_12_electricdevices_seed_00.model
baseline_model_RNN_data_13_epilepsy_seed_00.model
baseline_model_RNN_data_14_ering_seed_00.model
baseline_model_RNN_data_15_ethanolconcentration_seed_00.model
baseline_model_RNN_data_16_ethanollevel_seed_00.model
baseline_model_RNN_data_17_fish_seed_00.model
baseline_model_RNN_data_18_f

In [5]:
results.shape

torch.Size([80, 10, 2])

In [6]:
N_selected_seeds = 5
values, indices = torch.topk(results[:,:,0], N_selected_seeds, dim=1, largest=True, sorted=True)
selected_results = results.gather(1, indices.unsqueeze(-1).expand(-1, -1, 2))
selected_results.shape


torch.Size([80, 5, 2])

In [7]:
selected_results.mean(1)
np.savetxt(f"./evaluation/acc.txt", selected_results.mean(1).numpy(), delimiter="\t", fmt='%.5f')

In [8]:
rrr = selected_results.mean([1,2])
print(rrr)
sorted_data, sorted_indices = torch.sort(rrr, descending=True)

# Getting ranks
ranks = torch.argsort(sorted_indices)

print("Sorted tensor:", sorted_data)
print("Indices in original tensor:", sorted_indices)
print("Ranks:", ranks)

tensor([0.6780, 0.5250, 0.3290, 0.5420, 0.8792, 0.7830, 0.7390, 0.8031, 0.5800,
        0.8720, 0.8720, 0.4274, 0.4480, 0.5327, 0.6067, 0.3360, 0.3810, 0.4029,
        0.5980, 0.6460, 0.7960, 0.7690, 0.6100, 0.7060, 0.7857, 1.0000, 0.6929,
        0.7380, 0.3550, 0.1910, 0.9984, 0.9981, 0.9560, 0.4060, 0.4770, 0.7390,
        0.6280, 0.6250, 0.6180, 0.5750, 0.7990, 0.3843, 0.6440, 0.8071, 0.9431,
        0.8430, 0.7680, 0.6670, 0.4649, 0.3630, 0.3920, 0.7580, 0.4803, 0.5500,
        0.3450, 0.3500, 0.5125, 0.9370, 0.3970, 0.6383, 0.7690, 0.7970, 0.7840,
        0.6720, 0.6130, 0.4750, 0.5571, 0.5450, 0.6070, 0.5650, 0.4833, 0.4886,
        0.4500, 0.3670, 0.2290, 0.3230, 0.8900, 0.4319, 0.5801, 0.5390])
Sorted tensor: tensor([1.0000, 0.9984, 0.9981, 0.9560, 0.9431, 0.9370, 0.8900, 0.8792, 0.8720,
        0.8720, 0.8430, 0.8071, 0.8031, 0.7990, 0.7970, 0.7960, 0.7857, 0.7840,
        0.7830, 0.7690, 0.7690, 0.7680, 0.7580, 0.7390, 0.7390, 0.7380, 0.7060,
        0.6929, 0.6780, 0.6720, 

In [16]:
sorted_data[:40]

tensor([1.0000, 0.9984, 0.9981, 0.9560, 0.9431, 0.9370, 0.8900, 0.8792, 0.8720,
        0.8720, 0.8430, 0.8071, 0.8031, 0.7990, 0.7970, 0.7960, 0.7857, 0.7840,
        0.7830, 0.7690, 0.7690, 0.7680, 0.7580, 0.7390, 0.7390, 0.7380, 0.7060,
        0.6929, 0.6780, 0.6720, 0.6670, 0.6460, 0.6440, 0.6383, 0.6280, 0.6250,
        0.6180, 0.6130, 0.6100, 0.6070])

In [11]:
a = sorted_indices[:40].tolist()
a.sort()
a

[0,
 4,
 5,
 6,
 7,
 9,
 10,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 30,
 31,
 32,
 35,
 36,
 37,
 38,
 40,
 42,
 43,
 44,
 45,
 46,
 47,
 51,
 57,
 59,
 60,
 61,
 62,
 63,
 64,
 68,
 76]

In [12]:
temporal_datasets = ['Dataset_cbf.tsds',
                         'Dataset_chlorineconcentration.tsds',
                         'Dataset_cincecgtorso.tsds',
                         'Dataset_computers.tsds',
                         'Dataset_diatomsizereduction.tsds',
                         'Dataset_distalphalanxoutlineagegroup.tsds',
                         'Dataset_distalphalanxtw.tsds',
                         'Dataset_earthquakes.tsds',
                         'Dataset_ecg200.tsds',
                         'Dataset_ecg5000.tsds',
                         'Dataset_ecgfivedays.tsds',
                         'Dataset_eigenworms.tsds',
                         'Dataset_electricdevices.tsds',
                         'Dataset_epilepsy.tsds',
                         'Dataset_ering.tsds',
                         'Dataset_ethanolconcentration.tsds',
                         'Dataset_ethanollevel.tsds',
                         'Dataset_fish.tsds',
                         'Dataset_forda.tsds',
                         'Dataset_fordb.tsds',
                         'Dataset_freezerregulartrain.tsds',
                         'Dataset_freezersmalltrain.tsds',
                         'Dataset_gunpoint.tsds',
                         'Dataset_gunpointagespan.tsds',
                         'Dataset_gunpointmaleversusfemale.tsds',
                         'Dataset_gunpointoldversusyoung.tsds',
                         'Dataset_ham.tsds',
                         'Dataset_handoutlines.tsds',
                         'Dataset_haptics.tsds',
                         'Dataset_inlineskate.tsds',
                         'Dataset_insectepgregulartrain.tsds',
                         'Dataset_insectepgsmalltrain.tsds',
                         'Dataset_italypowerdemand.tsds',
                         'Dataset_largekitchenappliances.tsds',
                         'Dataset_mallat.tsds',
                         'Dataset_middlephalanxoutlineagegroup.tsds',
                         'Dataset_middlephalanxoutlinecorrect.tsds',
                         'Dataset_middlephalanxtw.tsds',
                         'Dataset_mixedshapesregulartrain.tsds',
                         'Dataset_mixedshapessmalltrain.tsds',
                         'Dataset_motestrain.tsds',
                         'Dataset_osuleaf.tsds',
                         'Dataset_phalangesoutlinescorrect.tsds',
                         'Dataset_plane.tsds',
                         'Dataset_powercons.tsds',
                         'Dataset_proximalphalanxoutlineagegroup.tsds',
                         'Dataset_proximalphalanxoutlinecorrect.tsds',
                         'Dataset_proximalphalanxtw.tsds',
                         'Dataset_racketsports.tsds',
                         'Dataset_refrigerationdevices.tsds',
                         'Dataset_screentype.tsds',
                         'Dataset_selfregulationscp1.tsds',
                         'Dataset_selfregulationscp2.tsds',
                         'Dataset_semghandgenderch2.tsds',
                         'Dataset_semghandmovementch2.tsds',
                         'Dataset_semghandsubjectch2.tsds',
                         'Dataset_shapeletsim.tsds',
                         'Dataset_slope.tsds',
                         'Dataset_smallkitchenappliances.tsds',
                         'Dataset_smoothsubspace.tsds',
                         'Dataset_sonyaiborobotsurface1.tsds',
                         'Dataset_sonyaiborobotsurface2.tsds',
                         'Dataset_starlightcurves.tsds',
                         'Dataset_strawberry.tsds',
                         'Dataset_symbols.tsds',
                         'Dataset_syntheticcontrol.tsds',
                         'Dataset_toesegmentation1.tsds',
                         'Dataset_trace.tsds',
                         'Dataset_twoleadecg.tsds',
                         'Dataset_twopatterns.tsds',
                         'Dataset_umd.tsds',
                         'Dataset_uwavegesturelibrary.tsds',
                         'Dataset_uwavegesturelibraryall.tsds',
                         'Dataset_uwavegesturelibraryx.tsds',
                         'Dataset_uwavegesturelibraryy.tsds',
                         'Dataset_uwavegesturelibraryz.tsds',
                         'Dataset_wafer.tsds',
                         'Dataset_worms.tsds',
                         'Dataset_wormstwoclass.tsds',
                         'Dataset_yoga.tsds']

In [14]:
new_ds = []
for idx in a:
    new_ds.append(temporal_datasets[idx])
new_ds

['Dataset_cbf.tsds',
 'Dataset_diatomsizereduction.tsds',
 'Dataset_distalphalanxoutlineagegroup.tsds',
 'Dataset_distalphalanxtw.tsds',
 'Dataset_earthquakes.tsds',
 'Dataset_ecg5000.tsds',
 'Dataset_ecgfivedays.tsds',
 'Dataset_fordb.tsds',
 'Dataset_freezerregulartrain.tsds',
 'Dataset_freezersmalltrain.tsds',
 'Dataset_gunpoint.tsds',
 'Dataset_gunpointagespan.tsds',
 'Dataset_gunpointmaleversusfemale.tsds',
 'Dataset_gunpointoldversusyoung.tsds',
 'Dataset_ham.tsds',
 'Dataset_handoutlines.tsds',
 'Dataset_insectepgregulartrain.tsds',
 'Dataset_insectepgsmalltrain.tsds',
 'Dataset_italypowerdemand.tsds',
 'Dataset_middlephalanxoutlineagegroup.tsds',
 'Dataset_middlephalanxoutlinecorrect.tsds',
 'Dataset_middlephalanxtw.tsds',
 'Dataset_mixedshapesregulartrain.tsds',
 'Dataset_motestrain.tsds',
 'Dataset_phalangesoutlinescorrect.tsds',
 'Dataset_plane.tsds',
 'Dataset_powercons.tsds',
 'Dataset_proximalphalanxoutlineagegroup.tsds',
 'Dataset_proximalphalanxoutlinecorrect.tsds',
 'D

In [18]:
len(temporal_datasets)

80

In [20]:
import os

# Replace with your specific directory
directory = "../dataset/"

# Assuming A and B are your lists of file names
A = ["file1.txt", "file2.txt", "file3.txt", ...]
B = ["file2.txt", "file3.txt", ...]

for file_name in temporal_datasets:
    if file_name not in new_ds:
        file_path = os.path.join(directory, file_name)
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"File {file_name} has been deleted.")
        else:
            print(f"File {file_name} does not exist.")

File Dataset_chlorineconcentration.tsds has been deleted.
File Dataset_cincecgtorso.tsds has been deleted.
File Dataset_computers.tsds has been deleted.
File Dataset_ecg200.tsds has been deleted.
File Dataset_eigenworms.tsds has been deleted.
File Dataset_electricdevices.tsds has been deleted.
File Dataset_epilepsy.tsds has been deleted.
File Dataset_ering.tsds has been deleted.
File Dataset_ethanolconcentration.tsds has been deleted.
File Dataset_ethanollevel.tsds has been deleted.
File Dataset_fish.tsds has been deleted.
File Dataset_forda.tsds has been deleted.
File Dataset_haptics.tsds has been deleted.
File Dataset_inlineskate.tsds has been deleted.
File Dataset_largekitchenappliances.tsds has been deleted.
File Dataset_mallat.tsds has been deleted.
File Dataset_mixedshapessmalltrain.tsds has been deleted.
File Dataset_osuleaf.tsds has been deleted.
File Dataset_racketsports.tsds has been deleted.
File Dataset_refrigerationdevices.tsds has been deleted.
File Dataset_screentype.tsd

In [9]:
m[4,:,2]

tensor([0.9091,    nan, 0.8377])

In [10]:
s[4,:,2]

tensor([0., nan, 0.])

In [11]:
m[5,:,2]

tensor([nan, nan, nan])

In [12]:
s[5,:,2]

tensor([nan, nan, nan])